# Preparing SumStats for GWAS Catalog

This notebook serves to inspect `regenie output of GWASToolKit`, and proceses the GWAS data for sharing through *GWAS Catalog*.


In [1]:
# pip install gwaslab==3.4.44


In [2]:
# conda install -c conda-forge polars

## Import necessary libraries

In [1]:
# Function to check for installation of required packages
def check_install_package(package_name):
    try:
        importlib.import_module(package_name)
    except ImportError:
        print(f'{package_name} is not installed. Installing it now...')
        subprocess.check_call(['pip', 'install', package_name])

import os
import glob
import importlib
import subprocess
import sys

# argument parsing
import argparse

# get date and time
from datetime import datetime

# Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool
check_install_package('pandas')
import pandas as pd

# pyarrow is supperior to loading parquet files
check_install_package('pyarrow')
import pyarrow as pa
import pyarrow.parquet as pq

# polars is a fast dataframe library
#check_install_package('polars')
#import polars as pl

# for statistical analysis
check_install_package('scipy')
from scipy import stats
import numpy as np

# scientific colourmaps
# https://www.fabiocrameri.ch/ws/media-library/8c4b111121ff448e843dfef9220bf613/readme_scientificcolourmaps.pdf
check_install_package('cmcrameri')
import cmcrameri as ccm
from cmcrameri import cm

# for plotting
check_install_package('matplotlib')
import matplotlib
import matplotlib.pyplot as plt
# if using a Jupyter notebook, include:
%matplotlib inline

# use Seaborn for visualisations
check_install_package('seaborn')
import seaborn as sns

# for handling GWAS data
import gwaslab as gl

from sklearn.metrics import jaccard_score

Creating directories for plots

In [ ]:
# Import necessary libraries
import os
from subprocess import check_output

# Set general defaults
POPULATION = "EUR"  # Population group
PHENOTYPE = "PHENOTYPE_NAME"  # Placeholder for phenotype name
REF_1KG = "1kg_eur_hg38"  # Reference data

# General plotting directory
PLOTS_loc = f"PLOTS/{PHENOTYPE}"

# Check if the general plotting directory exists, if not, create it
if not os.path.exists(PLOTS_loc):
    os.makedirs(PLOTS_loc)

# Regional association plots directory
REG_PLOTS_loc = os.path.join(PLOTS_loc, "Regional_Association_Plots")

# Check if the regional association plots directory exists, if not, create it
if not os.path.exists(REG_PLOTS_loc):
    os.makedirs(REG_PLOTS_loc)

# GWAS results directory - CHANGE to your specific path
GWAS_RES_loc = "/path/to/gwas/results"
print("Checking contents of the GWAS results directory:")
print(check_output(["ls", os.path.join(GWAS_RES_loc)]).decode("utf8"))

# GWAS datasets directory - CHANGE to your specific path
GWAS_DATASETS_loc = "/path/to/gwas/datasets"
print("Checking contents of the GWAS datasets directory:")
print(check_output(["ls", os.path.join(GWAS_DATASETS_loc)]).decode("utf8"))

# Check if the GWAS Catalog directory exists, if not, create it
if not os.path.exists("GWASCatalog"):
    os.makedirs("GWASCatalog")

# GWAS Catalog directory
GWASCatalog_loc = os.path.join("GWASCatalog")


## Loading data

Loading the different datasets.

In [5]:
# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output

# print(check_output(
#     ["ls", os.path.join(GWAS_RES_loc, SUBSTUDY_PHENO)]).decode("utf8"))

# carotid intima media thickness

We load in the GWAS on cIMT.

## Saving cIMT

Here we save the parsed data for future reference.

In [ ]:
# read in data
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe

import polars as pl

temp = pl.read_csv(
    source=os.path.join(
        GWAS_DATASETS_loc
        # + "/AEGS.GWAS.1kGp3v5GoNL5.CD68_counts_rankNorm.EXCL_DEFAULT.summary_results.txt.gz", #CHANGE!
        + "/regenie.IPH.summary_results.txt.gz", #CHANGE!
    ),
    has_header=True,
    separator=" ",
    ignore_errors=False,
    # n_rows=50000, # for debugging
    quote_char=None,
    # necessary to fix issues with missing values when reading data
    null_values=["NA"],
    # There is an error at import (from temp to pandas()):
    # Could not parse `X` as dtype `i64` at column 'CHR' (column number 2)
    # https://stackoverflow.com/questions/75797640/how-to-specify-column-types-in-python-polars-read-csv
    # https://stackoverflow.com/questions/71790235/switching-between-dtypes-within-a-dataframe
    # https://pola-rs.github.io/polars/user-guide/concepts/data-types/
    dtypes={"CHR": pl.Utf8},
)
# change polars dataframe to pandas dataframe
cd_norm_data = temp.to_pandas()
del temp

### Inspection of data

Here we do a quick check on what is what in the data.

#### Heads

Printing head of data.

In [ ]:
cd_norm_data.head()

#### Shapes

Printing shapes of data.

In [ ]:
print("Printing shape of data:\n", cd_norm_data.shape)

#### Columns

Printing columns of data.

In [ ]:
print("Printing columns of data:\n", cd_norm_data.columns)

#### Info

Printing info of Women-only data.

In [ ]:
print("Printing info of data:\n", cd_norm_data.info())

#### Column statistics

Getting some per column summary statistics of Women-only data.

In [ ]:
print("Printing describe of data:\n", cd_norm_data.describe())

### Basic visualisations

Here we plot histograms of allele frequencies, effect, and sample size .

#### Sampling from the data
Here we provide an example on how to take a sample of n=800,000 rows, representing ±10% of the data, for easy plotting. This should be representative for most things we are interested in.

In [12]:
# example code to get a sample

# cd_norm_data_sample = cd_norm_data.sample(800000)

In [ ]:
import seaborn as sns

sns.histplot(
    data=cd_norm_data,  # gwas_combo_sample,
    x="A1FREQ",
    bins=25,
    kde=False,
    stat="frequency",
    color="#1290D9",
)

# plt.savefig(
#     os.path.join(PLOTS_loc, "histogram.EAF." + POPULATION + ".png"),
#     dpi=300,
#     bbox_inches="tight",
#     format="png",
# )
plt.savefig(
    os.path.join(PLOTS_loc, "histogram.EAF." + PHENOTYPE + "_" + POPULATION + ".pdf"),
    dpi=300,
    bbox_inches="tight",
    format="pdf",
)

In [ ]:
sns.histplot(
    data=cd_norm_data,  # gwas_combo_sample,
    # x="BETA_FIXED",
    x="BETA",
    bins=25,
    kde=False,
    stat="frequency",
    color="#E55738",
)

# plt.savefig(
#     os.path.join(PLOTS_loc, "histogram.effect." + POPULATION + ".png"),
#     dpi=300,
#     bbox_inches="tight",
#     format="png",
# )
plt.savefig(
    os.path.join(PLOTS_loc, "histogram.effect." + PHENOTYPE + "_" + POPULATION + ".pdf"),
    dpi=300,
    bbox_inches="tight",
    format="pdf",
)

### SumStats quality control

Here we first check and fix headers, and contents of the SumStats object.

First, we load the data and inspect it using `GWASLab`. Note that for this to work, some column types need to be adjusted.

```
 0   SNP                object 
 1   CHR                object 
 2   POS                int64  
 3   HAPMAP_A1_FREQ     float64
 4   CODED_ALLELE       object 
 5   NONCODED_ALLELE    object 
 6   CODED_ALLELE_FREQ  float64
 7   N_EFF              float64
 8   P_SQRTN            float64
 9   BETA_FIXED         float64
 10  SE_FIXED           float64
 11  P_FIXED            float64
 12  P_RANDOM           float64
 13  DF                 int64  
 14  P_COCHRANS_Q       float64
 15  I_SQUARED          float64
 16  DIRECTIONS         object 
 17  GENES_1000KB       object 
 18  NEAREST_GENE       object 
 19  CAVEAT             object 
 ```

In [15]:
cd_norm_data['P'] = 10**(-cd_norm_data.LOG10P)

In [16]:
cd_norm_data[["CHROM"]] = cd_norm_data[["CHROM"]].astype("object")

In [17]:
cd_norm_data[["POS"]] = cd_norm_data[["GENPOS"]].astype("Int64")

In [18]:
# cd_norm_data['CAVEAT'].fillna('None', inplace=True)

In [19]:
# # create new SNPID column based on chromosome, position, and alleles
# # down the road we need an SNPID column to merge with the reference data and which does not contain 'ID' because this is not correctly interpreted by GWASLab
cd_norm_data["SNPID"] = (
    cd_norm_data["CHROM"].astype(str)
    + ":"
    + cd_norm_data["POS"].astype(str)
    + ":"
    + cd_norm_data["ALLELE0"].astype(str)
    + ":"
    + cd_norm_data["ALLELE1"].astype(str)
)

# # from an *.out file at /hpc/dhl_ec/svanderlaan/projects/sign/vonberg_joanna/2.Age.of.onset/linreg_aoo_unrestricted_bolt/amr.everything.butwhi/xx.group
# # cd_norm_data["N"] = 5344

In [20]:
cd_norm_data.rename(columns={"SNPID": "VariantID"}, inplace=True)

In [ ]:
cd_norm_data.head()

### Converting to GWASLab object

Here we are ready to load the data and inspect it using `GWASLab`. 

In [ ]:
import gwaslab as gl

# Specify the columns: - CHANGE!
cd_norm_data_sumstats = gl.Sumstats(
    cd_norm_data,
    snpid="VariantID",
    # rsid="RSID", # not available
    chrom="CHROM",
    pos="POS",
    ea="ALLELE1",
    nea="ALLELE0",
    eaf="A1FREQ",
    # maf="MAF",
    beta="BETA",
    se="SE",
    p="P",
    # direction="Direction",  # only for meta-GWAS
    n="N",
    # info="Info", # not available
    other=[
        # "MAC",
        # "MAF"
    #     "DF",
    #     "DIRECTIONS",
    #     "P_COCHRANS_Q",
    #     "I_SQUARED",
    #     "CAVEAT",
    ],
    build="38",
    verbose=True,
)

In [ ]:
cd_norm_data_sumstats.data

In [ ]:
cd_norm_data_sumstats.summary()

In [ ]:
cd_norm_data_sumstats.lookup_status()

#### Intermediate cleaning

Here we cleanup the originally loaded data, to clear memory.

In [26]:
del cd_norm_data

#### Get reference data

We align the data to the reference genome, this will work for most common variants. Before that, we check which reference datasets are available, and get these.

In [ ]:
# check references
gl.check_available_ref()

# {'1kg_eas_hg19': 'https://www.dropbox.com/s/lztaxqhy2o6dpxw/EAS.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?dl=1',
#  '1kg_eas_hg19_md5': 'c8c97434843c0da3113fc06879ead472',
#  '1kg_eas_hg19_tbi': 'https://www.dropbox.com/s/k9klefl8m9fcfo1/EAS.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?dl=1',
#  '1kg_eur_hg19': 'https://www.dropbox.com/s/1nbgqshknevseks/amr.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?dl=1',
#  '1kg_eur_hg19_md5': '734069d895009d38c2f962bfbb6fab52',
#  '1kg_eur_hg19_tbi': 'https://www.dropbox.com/s/vscvkrflh6fc5a0/amr.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?dl=1',
#  '1kg_eas_hg38': 'https://www.dropbox.com/s/3dstbbb1el9r3au/EAS.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?dl=1',
#  '1kg_eas_hg38_md5': 'f45e80bca9ef7b29e6b1832e6ac15375',
#  '1kg_eas_hg38_tbi': 'https://www.dropbox.com/s/vwnp5vd8dcqksn4/EAS.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?dl=1',
#  '1kg_eur_hg38': 'https://www.dropbox.com/s/z0mkehg17lryapv/amr.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?dl=1',
#  '1kg_eur_hg38_md5': '228d3285fa99132cc6321e2925e0768d',
#  '1kg_eur_hg38_tbi': 'https://www.dropbox.com/s/ze8g58x75x9qbf0/amr.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?dl=1',
#  '1kg_sas_hg19': 'https://www.dropbox.com/scl/fi/fubqvuj3p4ii4y35zknv8/SAS.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?rlkey=5z50f66iltjchcaszznq5bczt&dl=1',
#  '1kg_sas_hg19_md5': 'e2d3f9e2e6580d05e877e9effd435c4e',
#  '1kg_sas_hg19_tbi': 'https://www.dropbox.com/scl/fi/icnmrnzee7ofdpx5l96tg/SAS.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?rlkey=st8t88snby26q37rqi6zh5zck&dl=1',
#  '1kg_amr_hg19': 'https://www.dropbox.com/scl/fi/bxa4zfngsxsc38rhtiv8c/AMR.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?rlkey=ibcn8hb1n8n36j3u0jfzci267&dl=1',
#  '1kg_amr_hg19_md5': '68d3cdf01cbabdae6e74a07795fa881c',
#  '1kg_amr_hg19_tbi': 'https://www.dropbox.com/scl/fi/1zk16x7h4r89jurzwu05u/AMR.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?rlkey=b4cere4w38zvzyfitfge3r8n0&dl=1',
#  '1kg_sas_hg38': 'https://www.dropbox.com/scl/fi/jr3l5zz42py3kny2bccmj/SAS.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?rlkey=x0t6tsy71jxzf021wfqdn8k5q&dl=1',
#  '1kg_sas_hg38_md5': 'e5d79bea1958aa50c23f618d342ccc83',
#  '1kg_sas_hg38_tbi': 'https://www.dropbox.com/scl/fi/02oia4ur5r7w9qgiuf6i9/SAS.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?rlkey=00p9rxe0xzfs6hr1rg4d8oadm&dl=1',
#  '1kg_amr_hg38': 'https://www.dropbox.com/scl/fi/4t4tyuhzp78uyb6tgkroq/AMR.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?rlkey=p96gbs1tcdia31jnjv1b82kuz&dl=1',
#  '1kg_amr_hg38_md5': '229fbd610001cf6f137b7f738352a44a',
#  '1kg_amr_hg38_tbi': 'https://www.dropbox.com/scl/fi/x0dby543tr9xpaqj2i0ba/AMR.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?rlkey=uj8o7j0cy0spipe174jn54sqs&dl=1',
#  '1kg_afr_hg19': https://www.dropbox.com/scl/fi/tq4w9lyt5z47ym7grtrxg/AFR.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?rlkey=k3bimeu3yr5loq8hohba5mr6k&dl=1,
#  '1kg_afr_hg19_md5': 'f7b4425f39e8292dce6f13711e7f6c50',
#  '1kg_afr_hg19_tbi': 'https://www.dropbox.com/scl/fi/0giiptu0btwj1kfm6jdzr/AFR.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?rlkey=ucb5weprsc5prcg8hvtgmruxx&dl=1',
#  '1kg_pan_hg19': 'https://www.dropbox.com/scl/fi/6b4j9z9knmllfnbx86aw6/PAN.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz?rlkey=eento8vg06zyrkvooc9wd4cvu&dl=1',
#  '1kg_pan_hg19_md5': 'fed846482204487b60d33b21ddb18106',
#  '1kg_pan_hg19_tbi': 'https://www.dropbox.com/scl/fi/stco946scio5tvto0ln4j/PAN.ALL.split_norm_af.1kgp3v5.hg19.vcf.gz.tbi?rlkey=hfh53beb627lmqwv3d8mzqy0c&dl=1',
#  '1kg_afr_hg38': 'https://www.dropbox.com/scl/fi/239xmm7qijtnsks97chc9/AFR.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?rlkey=47en5fk1icbekpg7we3uot9g8&dl=1',
#  '1kg_afr_hg38_md5': '3bb7923be0809a324d7b7633b8d58a3b',
#  '1kg_afr_hg38_tbi': 'https://www.dropbox.com/scl/fi/3y3pg4yqwo2jaaamx1c8f/AFR.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?rlkey=say0ihfwa51z3otgn4bjtze8p&dl=1',
#  '1kg_pan_hg38': 'https://www.dropbox.com/scl/fi/nf01487smtmeq243ihfwm/PAN.ALL.split_norm_af.1kg_30x.hg38.vcf.gz?rlkey=3pefbkzxwcnejx4inynifpft7&dl=1',
#  '1kg_pan_hg38_md5': '23bb86d748c4a66e85e087f647e8b60e',
#  '1kg_pan_hg38_tbi': 'https://www.dropbox.com/scl/fi/hu7cttr4cenw5yjsm2775/PAN.ALL.split_norm_af.1kg_30x.hg38.vcf.gz.tbi?rlkey=568u7bkvkybm4wt2q9284o198&dl=1',
#  'dbsnp_v151_hg19': 'https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/VCF/00-All.vcf.gz',
#  'dbsnp_v151_hg19_tbi': 'https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/VCF/00-All.vcf.gz.tbi',
#  'dbsnp_v151_hg38': 'https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/00-All.vcf.gz',
#  'dbsnp_v151_hg38_tbi': 'https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/00-All.vcf.gz.tbi',
#  'dbsnp_v156_hg19': 'https://ftp.ncbi.nih.gov/snp/archive/b156/VCF/GCF_000001405.25.gz',
#  'dbsnp_v156_hg19_tbi': 'https://ftp.ncbi.nih.gov/snp/archive/b156/VCF/GCF_000001405.25.gz.tbi',
#  'dbsnp_v156_hg38': 'https://ftp.ncbi.nih.gov/snp/archive/b156/VCF/GCF_000001405.40.gz',
#  'dbsnp_v156_hg38_tbi': 'https://ftp.ncbi.nih.gov/snp/archive/b156/VCF/GCF_000001405.40.gz.tbi',
#  'ucsc_genome_hg19': 'http://hgdownload.cse.ucsc.edu/goldenpath/hg19/bigZips/hg19.fa.gz',
#  'ucsc_genome_hg38': 'https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz',
#  '1kg_dbsnp151_hg19_auto': 'https://www.dropbox.com/s/37p2u1xwmux4gwo/1kg_dbsnp151_hg19_auto.txt.gz?dl=1',
#  '1kg_dbsnp151_hg19_auto_md5': '7d1e7624fb6e4df7a2f6f05558d436b4',
#  '1kg_dbsnp151_hg38_auto': 'https://www.dropbox.com/s/ouf60n7gdz6cm0g/1kg_dbsnp151_hg38_auto.txt.gz?dl=1',
#  '1kg_dbsnp151_hg38_auto_md5': '4c7ef2d2415c18c286219e970fdda972',
#  'recombination_hg19': 'https://www.dropbox.com/s/wbesl8haxknonuc/recombination_hg19.tar.gz?dl=1',
#  'recombination_hg38': 'https://www.dropbox.com/s/vuo8mvqx0fpibzj/recombination_hg38.tar.gz?dl=1',
#  'ensembl_hg19_gtf': 'https://ftp.ensembl.org/pub/grch37/current/gtf/homo_sapiens/Homo_sapiens.GRCh37.87.chr.gtf.gz',
#  'ensembl_hg38_gtf': 'https://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens//Homo_sapiens.GRCh38.109.chr.gtf.gz',
#  'refseq_hg19_gtf': 'https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh37_latest/refseq_identifiers/GRCh37_latest_genomic.gtf.gz',
#  'refseq_hg38_gtf': 'https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.gtf.gz',
#  'testlink': 'https://www.dropbox.com/s/8u7capwge0ihshu/EAS.chr22.split_norm_af.1kgp3v5.vcf.gz?dl=1',
#  'testlink_tbi': 'https://www.dropbox.com/s/hdneg53t6u1j6ib/EAS.chr22.split_norm_af.1kgp3v5.vcf.gz.tbi?dl=1'}

In [28]:
# # download ref SNPID-rsID table first
# # hg19 is the same as GRCh37, which is the same as b37, which is the same as 19
# # USCSC Genome Browser hg19, SNP information
# gl.download_ref("ucsc_genome_hg19")
# # combined 1KG and dbSNP151, hg19, autosomes
# gl.download_ref("1kg_dbsnp151_hg19_auto")
# # gl.download_ref("refseq_hg19_gtf") # gene annotation, hg19

In [29]:
# gl.download_ref("dbsnp_v156_hg19")

In [30]:
# gl.download_ref("dbsnp_v156_hg19_tbi")

In [31]:
# download_ref
# gl.download_ref(REF_1KG)

In [32]:
# download_ref("recombination_hg19") - get recombination map for hg19
# gl.download_ref("recombination_hg19")

In [33]:
# download_ref("ensembl_hg19_gtf") - get the Ensembl GTF file for hg19
# gl.download_ref("ensembl_hg19_gtf")

In [34]:
# download_ref("refseq_hg19_gtf") - get the refseq hg19 gtf file
# gl.download_ref("refseq_hg19_gtf")

### Basic check, harmonization, normalization, quality control, and filtering

Here we fix the dataset. 

#### Basic check

Here we apply basic check to the data, which makes sure that:
- SNPIDs are of the form chr:bp
- orders the data
- all alleles are capitalized
- does sanity checks on data

However, no data is filtered, and normalization (to a reference) is also not applied!

In [ ]:
# Execute `bacis_check` function - first we just make sure the data has the expected format, columns, and datatypes.
# full data
cd_norm_data_sumstats.basic_check(verbose=True)

In [ ]:
# check the form of the data as it is AFTER `basic_check` function
# full data
cd_norm_data_sumstats.data

In [ ]:
cd_norm_data_sumstats.summary()

In [ ]:
cd_norm_data_sumstats.lookup_status()

#### Remove duplicate and multi-allelic variants

Here we remove duplicate and multi-allelic variants, and keep the variants with the lowest P-value for association.

In [39]:
# removing multiallelic and duplicate variants
# mode=d ,remove duplicate.
#     remove duplicate SNPs based on 1. SNPID,
#     remove duplicate SNPs based on 2. CHR, POS, EA, and NEA
#     remove duplicate SNPs based on 3. rsID
# mode=m, remove multiallelic variants.
#     remove multiallelic SNPs based on 4. CHR, POS
# remove=True : remove NAs
# keep_col : use which column to sort the values (keep_ascend=True: ascending order)
# keep: keep 'first' or 'last'.

# gwas_combo_sumstats_sample.remove_dup(
#     mode="md",  # remove multi-allelic and duplicate variants
#     remove=True,  # remove NAs
#     keep_col="P",
#     keep_ascend=True,
#     keep="first",  # keep the first variant, with the lowest p-value (sorted by that column)
# )

In [ ]:
# # full dataset
cd_norm_data_sumstats.remove_dup(
    mode="md",  # remove multi-allelic and duplicate variants
    remove=False,  # remove NAs
    keep_col="P",
    keep_ascend=True,
    # keep the first variant, with the lowest p-value (sorted by that column)
    keep="first",
)

#### Harmonization

Here we harmonize the data with the reference:
- we make sure alleles are oriented according to the reference
- we assign rsIDs
- we flip alleles (and effect sizes), when necessary

##### Align NEA with REF in the reference genome

We check if the non-effect allele is aligned with the reference sequence (`hg19`). The status code will be changed accordingly.

In [ ]:
# # .check_ref(): Check if NEA is aligned with the reference sequence. After checking, the tracking status code will be changed accordingly.

# # full dataset
cd_norm_data_sumstats.check_ref(
    ref_seq=gl.get_path("ucsc_genome_hg38"), # ucsc_genome_hg38 - ucsc_genome_hg19
    #   chr_dict=gl.get_number_to_NC(build="19")
)

In [ ]:
# we make sure to flipp the alleles based on the status code

# full dataset
cd_norm_data_sumstats.flip_allele_stats()

In [ ]:
cd_norm_data_sumstats.lookup_status()

##### Check palindromic SNPs or indistinguishable Indels

Here we check for palindromic variants and indistinguishable INDELs and remove these. 


In [44]:
# .infer_strand() - skipped as this is done
# ref_infer= gl.get_path("1kg_AFR_hg19"), # reference vcf file
# ref_alt_freq=None,
# maf_threshold=0.40, # maf threshold for strand inference
# remove_snp="", # remove snps from the data
# mode="pi",
# n_cores=1,
# remove_indel=""

# first we check the strand of the data and assign the status code
# gwas_combo_sumstats_sample.infer_strand(ref_infer=gl.get_path("1kg_AFR_hg19"), n_cores=6)

# full data
# gwas_combo_sumstats.infer_strand(ref_infer = gl.get_path("1kg_AFR_hg19"),
#                                        n_cores=6)
# then we flip the alleles according to the status code
# gwas_combo_sumstats_sample.flip_allele_stats()

In [ ]:
# Infer the strand of the variants in the summary statistics data
# This function checks the strand of the data and assigns the status code accordingly
# The reference VCF file and the reference allele frequency column are specified
# The number of cores to use for parallel processing is also specified

# Define the reference VCF file path and allele frequency column
ref_vcf_path = gl.get_path(REF_1KG)  # Use the reference genome specified earlier
ref_alt_freq_col = "AF"  # Column name for allele frequency in the reference VCF

# Infer the strand of the variants
cd_norm_data_sumstats.infer_strand(ref_infer=ref_vcf_path, ref_alt_freq=ref_alt_freq_col, n_cores=8)

In [ ]:
cd_norm_data_sumstats.flip_allele_stats()

##### Assign rsID to the data

For the sample dataset this will take not much time. For a dataset with 8M variants it will take about 50-60 minutes! We should fix the SNPIDs such that it includes the alleles. This way the assigning of rsID will go smoothly. 

In [47]:
# rsID annotation based on chr, pos, ref, alt using a VCF file when the SNPID is of the form chr:pos.
# .assign_rsid() options	DataType	Description	Default
# ref_rsid_tsv	string	tsv file path for annotation of commonly used variants using SNPID (like 1:725932:G:A) as key.	-
# ref_rsid_vcf	string	vcf file path for annotation of other variants. .tbi file is also needed.	-
# chr_dict	dict	a dictionary for converting 1-25 to CHR in the vcf files.
# For example, the notation in dbSNP vcf file is based on RefSeq (like NC_000001.10).
# gwaslab provides built-in conversion dictionaries.
# gl.get_number_to_NC(build="19")
# n_cores	int	number of cores to use.

# gwas_combo_sumstats_sample.assign_rsid(
#     n_cores=4,
#     # ref_rsid_tsv = gl.get_path("1kg_dbsnp151_hg19_auto"),
#     ref_rsid_vcf="/Users/slaan3/PLINK/references/dbSNP/GCF_000001405.25.vcf.gz",  # this works when SNPID is in the format chr:pos
#     chr_dict=gl.get_number_to_NC(
#         build="19"
#     ),  # this is needed as in the VCF file, the chromosome is in NC format
# )

In [ ]:
cd_norm_data_sumstats.assign_rsid(
    n_cores=4,
    ref_rsid_tsv = gl.get_path("1kg_dbsnp151_hg38_auto"),
    #CHANGE!
    # ref_rsid_vcf="/tpeters/references/GCF_000001405.25.gz",  # this works when SNPID is in the format chr:pos
    chr_dict=gl.get_number_to_NC(
        build="38" # was 19
    ),  # this is needed as in the VCF file, the chromosome is in NC format
)

In [ ]:
cd_norm_data_sumstats.fix_id(
    fixid=True,
    forcefixid=True,
    overwrite=True,
)


In [ ]:
cd_norm_data_sumstats.data

In [51]:
# cd_norm_data_sumstats.assign_rsid(
#     n_cores=8,
#     # this works for common variants
#     ref_rsid_tsv=gl.get_path("1kg_dbsnp151_hg19_auto"),
#     # ref_rsid_vcf=gl.get_path("dbsnp_v156_hg19"),
#     chr_dict=gl.get_number_to_NC(
#         build="19"
#     ),  # this is needed as in the VCF file, the chromosome is in NC format
# )

In [ ]:
cd_norm_data_sumstats.summary()

In [ ]:
cd_norm_data_sumstats.lookup_status()

##### Plot frequencies of data against the reference

We would like to plot the frequency of the GWAS data compared to the reference data. We do this on a random subset of the data (n=200,000) as this is more efficient and should be representative of the full datasets.

In [ ]:
# cd_norm_data_sumstats_sample = cd_norm_data_sumstats_qc.random_variants(n=400000)
# cd_norm_data_sumstats_sample = cd_norm_data_sumstats.random_variants(n=400000)

In [ ]:
# cd_norm_data_sumstats_sample.check_af(
#      ref_infer=gl.get_path(REF_1KG),
#     ref_alt_freq="AF",
#     n_cores=8,
# )

In [ ]:
# cd_norm_data_sumstats_sample.plot_daf(threshold=0.12)

In [ ]:
# Check allele frequencies against the reference genome
# This function compares the allele frequencies in the summary statistics data with those in the reference genome
# The reference VCF file and the reference allele frequency column are specified
# The number of cores to use for parallel processing is also specified

# Define the reference VCF file path and allele frequency column
ref_vcf_path = gl.get_path(REF_1KG)  # Use the reference genome specified earlier
ref_alt_freq_col = "AF"  # Column name for allele frequency in the reference VCF

# Check allele frequencies
cd_norm_data_sumstats.check_af(
    ref_infer=ref_vcf_path,  # Path to the reference VCF file
    ref_alt_freq=ref_alt_freq_col,  # Column name for allele frequency in the reference VCF
    n_cores=8,  # Number of cores to use for parallel processing
)

In [ ]:
cd_norm_data_sumstats.plot_daf(threshold=0.12)

In [58]:
# cd_norm_data_sumstats_qc.plot_daf(threshold=0.12)

In [59]:
# del cd_norm_data_sumstats_sample

##### Summary

So in summary we have done:
- a basic check, making sure the chr, pos, alleles are all in order
- multi-allelic, weird chromosomes, and duplicate variants are removed
- data are harmonized, such that the rsID is added, the alleles are relative to the references (and flipped if needed) 

> Note, while we do calculate the difference in allele frequencies between the datasets and the reference, we **do not** filter on this for the purpose of sharing through _GWAS Catalog_. 


In [60]:
# print("The parsed and harmonized SumStats object:\n")
# 
# cd_norm_data_sumstats.data

In [61]:
#  cd_norm_data_sumstats.data["CAVEAT"].unique()

In [62]:
# temp = cd_norm_data_sumstats.data["CAVEAT"].value_counts()

# temp.to_csv(
#     os.path.join(
#         GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".counts_caveats.csv",
#     )
# )

### Save or open data
Let's save (or open) the summary statistics object for future reference.

In [ ]:
import gwaslab as gl

gl.dump_pickle(
    cd_norm_data_sumstats,
    os.path.join(
        GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.pkl",
    ),
    overwrite=True,
)

In [ ]:
# for future reference to open the pickle file

cd_norm_data_sumstats = gl.load_pickle(
    os.path.join(
        GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.pkl",
    )
)

In [6]:
FUMA_loc = os.path.join(GWASCatalog_loc+"/FUMA/")

# Check if the directory exists
if not os.path.exists(FUMA_loc):
    # If it doesn't exist, create it
    os.makedirs(FUMA_loc)

In [ ]:
# Specify the output location for the text file
output_file = os.path.join(
    FUMA_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.txt"
)

# Use the write method to export the sumstat data to a text file 
cd_norm_data_sumstats.to_format(output_file, fmt="fastgwa")

In [ ]:
import gwaslab as gl

cd_norm_data_sumstats.log.show()

cd_norm_data_sumstats.log.save(
    os.path.join(
        GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.log",
    )
)


LOAD PICKLE FILES

In [64]:
# cd_norm_data_sumstats_qc = gl.load_pickle(
#     os.path.join(
#         GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b37.gwaslab.qc.pkl",
#     )
# )

##### Parquet

Save it as a parquet for easy loading as a dataframe in other programs (_e.g._ `R`).

In [65]:
# import pyarrow as pa
# import pyarrow.parquet as pq

# # Convert DataFrame to Apache Arrow Table
# temp_table = pa.Table.from_pandas(cd_norm_data_sumstats.data)

# # Parquet with Brotli compression
# pq.write_table(
#     temp_table,
#     os.path.join(
#         GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b37.gwaslab.parquet",
#     ),
#     compression="BROTLI",
# )
# # we delete the temporary table object
# del temp_table

##### GWAS Catalog

Save it in `GWAS catalog`-format

In [66]:
# cd_norm_data_sumstats.to_format(
#     os.path.join(GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b37.gwaslab"),
#     fmt="ssf",
#     build="19",
# )

After this, we need to validate results for `GWAS Catalog`. For this purpose we created an environment based on python 3.9 and used `gwas-sumstats-tools` (https://github.com/EBISPOT/gwas-sumstats-tools). We remove any remaining variant with alleles formatted such that is unacceptable for `GWAS Catalog`.

### Filtering

Let's filter based on the following:

- CAVEAT == None

In [67]:
# cd_norm_data_sumstats.data.dtypes

In [ ]:
# cd_norm_data_sumstats_qc = cd_norm_data_sumstats.filter_value(
#     '(EAF>=0.01 & EAF<0.99 & DF>=1)'
# )
cd_norm_data_sumstats_qc = cd_norm_data_sumstats.filter_value(
    '(DAF>=-0.12 & DAF<0.12)'
)

# cd_norm_data_sumstats_qc = cd_norm_data_sumstats

In [ ]:
cd_norm_data_sumstats_qc.data

In [ ]:
cd_norm_data_sumstats_qc.summary()

In [ ]:
cd_norm_data_sumstats_qc.lookup_status()

In [ ]:
import gwaslab as gl

gl.dump_pickle(
    cd_norm_data_sumstats_qc,
    os.path.join(
        GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.qc.001.pkl",
    ),
    overwrite=True,
)

In [73]:
# cd_norm_data_sumstats_qc.to_format(
#     os.path.join(GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b37.qc.001.gwaslab"),
#     fmt="ssf",
#     build="19",
# )

In [ ]:
# cd_norm_data_sumstats_qc = gl.load_pickle(
#     os.path.join(
#         GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.qc.001.pkl",
#     )
# )

In [ ]:
# cd_norm_data_sumstats_qc = gl.load_pickle(
#     os.path.join(
#         GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.2.gwaslab.qc.001.pkl",
#     )
# )

### Visualisations

Here we attempt to create a genome-wide plot annotating significant genes.

#### Manhattan and stratified QQ plots

Here we create an automatically annotated Manhattan plot. Annotation is based on hg19 (GRCh37), ensembl 87 (https://ftp.ensembl.org/pub/grch37/release-109/gtf/homo_sapiens/).



In [ ]:
# # manhattan and qq plot
cd_norm_data_sumstats.plot_mqq(
    skip=2,
    cut=10,
    sig_line=True,
    sig_level=5e-8,
    anno="GENENAME",
    anno_style="right",
    windowsizekb=500,
    arm_offset=2,
    repel_force=0.02,  # default 0.01
    use_rank=True,
    build="38",
    # mode="m",
    stratified=True,
    drop_chr_start=True,
    figargs={"figsize": (20, 8), "dpi": 300},
    title="" + PHENOTYPE + "_" + POPULATION + "",
    save=os.path.join(
        PLOTS_loc, "manhattan.500kb.300dpi." + PHENOTYPE + "_" + POPULATION + ".pdf"),
    saveargs={"dpi": 300},
    verbose=True,
    marker_size=(25,25)
)

In [ ]:
# manhattan and qq plot
cd_norm_data_sumstats_qc.plot_mqq(
    skip=2,
    cut=10,
    sig_line=True,
    sig_level=5e-8,
    anno="GENENAME",
    anno_style="right",
    windowsizekb=500,
    arm_offset=2,
    repel_force=0.02,  # default 0.01
    use_rank=True,
    build="38",
    # mode="m",
    stratified=True,
    drop_chr_start=True,
    figargs={"figsize": (20, 8), "dpi": 300},
   title="" + PHENOTYPE + "_" + POPULATION + "_" + "QC" + "",
    save=os.path.join(
        PLOTS_loc, "manhattan.500kb.300dpi." + PHENOTYPE + "_" + POPULATION + ".QC.pdf"),
    saveargs={"dpi": 300},
    verbose=True,
    marker_size=(25,25)
)

### Top loci

We inventory the top loci. 


In [ ]:
cd_norm_data_sumstats.get_lead(anno=True, sig_level=5e-8, verbose=True)

In [ ]:
top_qc_loci = cd_norm_data_sumstats_qc.get_lead(anno=True, sig_level=5e-6, verbose=True)
top_qc_loci

In [ ]:
for i, row in top_qc_loci.iterrows():
    print(f"Regional plot for gene {row['GENE']}")
    min_pos = row["POS"] - 1000000
    max_pos = row["POS"] + 1000000
    if min_pos < 0:
        min_pos = 0
    cd_norm_data_sumstats_qc.plot_mqq(mode="r",build="38", region=(row["CHR"],min_pos,max_pos),region_grid=True, figargs={"figsize": (25, 15), "dpi": 300},
                                      vcf_path=gl.get_path("1kg_eur_hg38"),
                                      title= row["GENE"] + "_" + PHENOTYPE + "_" + POPULATION + "_" + "QC",
                                      save=os.path.join(REG_PLOTS_loc, "regional.plot.chr" + str(row["CHR"]) + ".pos" + str(row["POS"]) + "." + row["GENE"] + ".300dpi." + PHENOTYPE + "_" + POPULATION + ".QC.pdf"), 
                                      saveargs={"dpi": 300},)

In [89]:
del cd_norm_data_sumstats_qc, cd_norm_data_sumstats_qc2

### LOAD not-normalized data

In [ ]:
cd_norm_data_sumstats_qc = gl.load_pickle(
    os.path.join(
        GWASCatalog_loc + "/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab.qc.001.pkl",
    )
)

In [ ]:
cd_norm_data_sumstats_qc2 = gl.load_pickle("/Users/tpeters4/Library/CloudStorage/OneDrive-UMCUtrecht/Documenten/PhD Documents/Projecten/GWAS/Post_regenie_pgen/GWASCatalog/META_IPH_CLAM_EUR.b38.gwaslab.qc.001.pkl")

In [ ]:
# Specify the output location for the text file
output_file = os.path.join(
    GWASCatalog_loc + "/GWAS_SSF/META_" + PHENOTYPE + "_" + POPULATION + ".b38.gwaslab"
)

# Use the write method to export the sumstat data to a text file 
cd_norm_data_sumstats_qc.to_format(output_file, fmt="ssf")

In [ ]:
# Specify the output location for the text file
output_file = os.path.join(
    GWASCatalog_loc + "/GWAS_SSF/META_IPH_CLAM_" + POPULATION + ".b38.gwaslab"
)

# Use the write method to export the sumstat data to a text file 
cd_norm_data_sumstats_qc2.to_format(output_file, fmt="ssf")

In [ ]:
# Get the lead SNPs from the summary statistics data
# This function identifies the lead SNPs that are genome-wide significant
# anno=True adds annotation to the lead SNPs
# sig_level=5e-5 sets the significance threshold
# verbose=True enables verbose output
top_qc_loci = cd_norm_data_sumstats_qc.get_lead(anno=True, sig_level=5e-5, verbose=True)

# Display the top loci
top_qc_loci

In [ ]:
# Get the lead SNPs from the second summary statistics data
# This function identifies the lead SNPs that are genome-wide significant
# anno=True adds annotation to the lead SNPs
# sig_level=5e-5 sets the significance threshold
# verbose=True enables verbose output
top_qc_loci2 = cd_norm_data_sumstats_qc2.get_lead(anno=True, sig_level=5e-5, verbose=True)

# Display the top loci
top_qc_loci2

In [ ]:
# Extract the sets of rsIDs from the top loci of both datasets
set1 = set(top_qc_loci["rsID"])
set2 = set(top_qc_loci2["rsID"])

# Print the sets of rsIDs
print("Set of rsIDs from the first dataset:", set1)
print("Set of rsIDs from the second dataset:", set2)

In [7]:
lookup_genes = ["CCXL1", "CCXL3", "F5", "HMOX1", "CCR1", "CCR2", "MMP9", "FOS", "TIMP3"]

In [ ]:
import requests

def get_gene_coordinates(gene_symbols):
    """
    Fetches the genomic coordinates for a list of gene symbols using the Ensembl REST API.

    Parameters:
    gene_symbols (list): List of gene symbols to fetch coordinates for.

    Returns:
    dict: A dictionary where keys are gene symbols and values are dictionaries with chromosome, start, and end positions.
    """
    gene_to_coords = {}
    url_template = "https://rest.ensembl.org/lookup/symbol/homo_sapiens/{}?content-type=application/json"
    
    for gene in gene_symbols:
        response = requests.get(url_template.format(gene))
        if response.ok:
            data = response.json()
            gene_to_coords[gene] = {
                "chrom": data["seq_region_name"],
                "start": data["start"],
                "end": data["end"]
            }
        else:
            print(f"Could not fetch coordinates for gene: {gene}")
    
    return gene_to_coords

# Example usage
lookup_genes = ["CCXL1", "CCXL3", "F5", "HMOX1", "CCR1", "CCR2", "MMP9", "FOS", "TIMP3"]
gene_coords = get_gene_coordinates(lookup_genes)
print(gene_coords)

In [9]:
df1_snps = cd_norm_data_sumstats_qc.data

In [ ]:
# Initialize an empty list to store the best SNPs for each gene
results_best_snp1 = []

# Iterate over each gene in the gene coordinates dictionary
for gene in gene_coords:
    print(gene)
    # Extract chromosome, start, and end positions for the current gene
    chrom, start, end = gene_coords[gene]["chrom"], gene_coords[gene]["start"], gene_coords[gene]["end"]
    print(f'chrom: {chrom} - start: {start} - end: {end}')
    
    # Filter SNPs that fall within the current gene's chromosomal range
    filtered_snps = df1_snps[(df1_snps["CHR"] == int(chrom)) & 
                             (df1_snps["POS"] >= int(start)) & 
                             (df1_snps["POS"] <= int(end))]
    
    # If there are SNPs in the filtered range, find the one with the lowest p-value
    if not filtered_snps.empty:
        best_snp = filtered_snps.loc[filtered_snps["P"].idxmin()]
        # Append the best SNP information to the results list
        results_best_snp1.append({
            "gene": gene, 
            "CHR": chrom, 
            "POS": best_snp["POS"], 
            "EA": best_snp["EA"], 
            "NEA": best_snp["NEA"], 
            "SNPID": best_snp["SNPID"], 
            "P": best_snp["P"], 
            "rsID": best_snp["rsID"]
        })

# Print the results
print(results_best_snp1)

In [ ]:
# Initialize an empty list to store the set of rsIDs
set1 = []

# Iterate over each SNP in the results_best_snp1 list
for snp in results_best_snp1:
    # Append the rsID of the SNP to the set1 list
    set1.append(snp["rsID"])

# Print the set of rsIDs
print(set1)

In [26]:
df2_snps = cd_norm_data_sumstats_qc2.data

In [ ]:
# Initialize an empty list to store the best SNPs for each gene
results_best_snp2 = []

# Iterate over each gene in the gene coordinates dictionary
for gene in gene_coords:
    print(gene)
    # Extract chromosome, start, and end positions for the current gene
    chrom, start, end = gene_coords[gene]["chrom"], gene_coords[gene]["start"], gene_coords[gene]["end"]
    print(f'chrom: {chrom} - start: {start} - end: {end}')
    
    # Filter SNPs that fall within the current gene's chromosomal range
    filtered_snps = df2_snps[(df2_snps["CHR"] == int(chrom)) & 
                             (df2_snps["POS"] >= int(start)) & 
                             (df2_snps["POS"] <= int(end))]
    
    # If there are SNPs in the filtered range, find the one with the lowest p-value
    if not filtered_snps.empty:
        best_snp = filtered_snps.loc[filtered_snps["P"].idxmin()]
        # Append the best SNP information to the results list
        results_best_snp2.append({
            "gene": gene, 
            "CHR": chrom, 
            "POS": best_snp["POS"], 
            "EA": best_snp["EA"], 
            "NEA": best_snp["NEA"], 
            "SNPID": best_snp["SNPID"], 
            "P": best_snp["P"], 
            "rsID": best_snp["rsID"]
        })

# Convert the results to a DataFrame and print
print(results_best_snp2)

In [ ]:
# Initialize an empty list to store the set of rsIDs
set2 = []

# Iterate over each SNP in the results_best_snp2 list
for snp in results_best_snp2:
    # Append the rsID of the SNP to the set2 list
    set2.append(snp["rsID"])

# Print the set of rsIDs
print(set2)


In [78]:
# common_elements = ["CCXL1", "CCXL3", "F5", "HMOX1", "CCR1", "CCR2", "MMP9", "FOS", "TIMP3"]
# common_elements = ["CCXL1", "CCXL3", "F5", "HMOX1", "CCR1", "CCR2", "MMP9", "FOS", "TIMP3"]

# print(common_elements)
# set1 = top_qc_loci[top_qc_loci["GENE"].isin(common_elements)]["rsID"].values
# set2 = top_qc_loci2[top_qc_loci2["GENE"].isin(common_elements)]["rsID"].values

# print(set1)
# print(set2)

In [ ]:
# Find common genes between the two sets of top loci
common_elements = list(set(top_qc_loci["GENE"]).intersection(set(top_qc_loci2["GENE"])))

# Print the common genes
print("Common genes:", common_elements)

# Extract the rsIDs of the SNPs associated with the common genes from the first dataset
highlight_1 = top_qc_loci[top_qc_loci["GENE"].isin(common_elements)]["rsID"].values

# Extract the rsIDs of the SNPs associated with the common genes from the second dataset
highlight_2 = top_qc_loci2[top_qc_loci2["GENE"].isin(common_elements)]["rsID"].values

# Print the rsIDs from both datasets
print("rsIDs from the first dataset:", highlight_1)
print("rsIDs from the second dataset:", highlight_2)

### MIAMI

In [ ]:
cd_vs_norm_sumstats = gl.plot_miami2(
    path1=cd_norm_data_sumstats_qc,
    path2=cd_norm_data_sumstats_qc2,
    sig_level=5e-8,
    # sig_line_color="blue",
    # suggestive_sig_line = True,
    # suggestive_sig_line_color="blue",
    # additional_line=[5e-6],
    # additional_line_color=["grey"],
    titles=[PHENOTYPE, "IPH_Model"],
    # titles_pad=[0.1, 0.1],
    # titles_pad_adjusted=[0.1, 0.1],
    id1="rsID",
    id2="rsID",
    windowsizekb=500,
    arm_offset=2,
    repel_force=0.02,  # default 0.01
    build="38",
    # anno_set=[(16, 83035073)], #CHANGE! - select significant snps/ top loci
    # highlight=[(16, 83035073)], # CHANGE! - significant snps
    highlight_windowkb=500,
    save=os.path.join(PLOTS_loc, "miami.500kb.300dpi.no_genes." + PHENOTYPE + "_vs_IPH_Model.pdf"),
    save_args={"dpi": 300},
    verbose=True
)

In [ ]:
cd_vs_norm_sumstats = gl.plot_miami2(
    path1=cd_norm_data_sumstats_qc,
    path2=cd_norm_data_sumstats_qc2,
    sig_level=5e-8,
    # sig_line_color="blue",
    # suggestive_sig_line = True,
    # suggestive_sig_line_color="blue",
    # additional_line=[5e-8],
    # additional_line_color=["grey"],
    titles=[PHENOTYPE, "IPH_Model"],
    # titles_pad=[0.1, 0.1],
    # titles_pad_adjusted=[0.1, 0.1],
    id1="rsID",
    id2="rsID",
    anno1="GENENAME",
    anno_style1="right",
    anno_set1=list(set1),
    anno2="GENENAME",
    anno_style2="right",
    anno_set2=list(set2),
    highlight1=list(set1),
    highlight2=list(set2),
    windowsizekb=500,
    arm_offset=2,
    repel_force=0.02,  # default 0.01
    build="38",
    # anno_set=[(16, 83035073)], #CHANGE! - select significant snps/ top loci
    # highlight=[(16, 83035073)], # CHANGE! - significant snps
    highlight_windowkb=500,
    save=os.path.join(PLOTS_loc, "miami.500kb.300dpi.manual_genes." + PHENOTYPE + "_vs_IPH_Model.pdf"),
    save_args={"dpi": 300},
    verbose=True
)

In [ ]:
cd_vs_norm_sumstats = gl.plot_miami2(
    path1=cd_norm_data_sumstats_qc,
    path2=cd_norm_data_sumstats_qc2,
    sig_level=5e-6,
    sig_line_color="blue",
    # suggestive_sig_line = True,
    # suggestive_sig_line_color="blue",
    additional_line=[5e-8],
    additional_line_color=["grey"],
    titles=[PHENOTYPE, "IPH_Model"],
    # titles_pad=[0.1, 0.1],
    # titles_pad_adjusted=[0.1, 0.1],
    id1="rsID",
    id2="rsID",
    anno1="GENENAME",
    anno_style1="right",
    anno_set1=list(set1),
    anno2="GENENAME",
    anno_style2="right",
    anno_set2=list(set2),
    highlight1=higlight_1,
    highlight2=higlight_2,
    windowsizekb=500,
    arm_offset=2,
    repel_force=0.02,  # default 0.01
    build="38",
    # anno_set=[(16, 83035073)], #CHANGE! - select significant snps/ top loci
    # highlight=[(16, 83035073)], # CHANGE! - significant snps
    highlight_windowkb=500,
    save=os.path.join(PLOTS_loc, "miami.500kb.300dpi.genes." + PHENOTYPE + "_vs_IPH_Model.pdf"),
    save_args={"dpi": 300},
    verbose=True
)

### EFFECT SIZE
Compare effect size between phenotypes

In [ ]:
gl.compare_effect(
    cd_norm_data_sumstats_qc,
    cd_norm_data_sumstats_qc2,
    mode="beta",
    label=[PHENOTYPE,"IPH CLAM","Both","None"],
    sig_level=5e-6,
    legend_title=r"$ P < 5 x 10^{-8}$ in:",
    legend_title2=r"Heterogeneity test:",
    legend_pos="upper left",
    #  legend_args=None,
    xylabel_prefix="Per-allele effect size in ",
    is_reg=True,
    is_45_helper_line=True,
    anno=True,
    # anno_min=0,
    # anno_min1=0,
    # anno_min2=0,
    # anno_diff=0,
    # is_q=False,
    q_level=0.05,
    # anno_het=False,
    # r_se=False,
    #  fdr=False,
    #  legend_mode="full",
     save=True,
    # saveargs=None,
    verbose=False,
)

In [58]:
set1 = top_qc_loci[["SNPID", "GENE"]]
set2 = top_qc_loci2[["SNPID", "GENE"]]

# find the union of the sets and converting resultant set to list
suggestive_snps = pd.concat([set1, set2])

In [ ]:
suggestive_snps

In [11]:
# Convert the summary statistics objects to pandas DataFrames
df1 = cd_norm_data_sumstats_qc.data
df2 = cd_norm_data_sumstats_qc2.data

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
print(f'lenght df1: {len(df1)}')
print(f'lenght df2: {len(df2)}')

In [15]:
# Extract beta values and SNP IDs from both datasets
beta1_df = cd_norm_data_sumstats_qc.data[["rsID", "BETA", "P", "EAF", "SNPID"]].rename(columns={"BETA": "BETA1", "P": "P1", "EAF": "EAF1"})
beta2_df = cd_norm_data_sumstats_qc2.data[["rsID", "BETA", "P", "EAF"]].rename(columns={"BETA": "BETA2", "P": "P2", "EAF": "EAF2"})

In [ ]:
duplicates_beta1 = beta1_df[beta1_df.duplicated(subset="rsID", keep=False)]
duplicates_beta2 = beta2_df[beta2_df.duplicated(subset="rsID", keep=False)]

print(f"Number of duplicate rsID in beta1_df: {len(duplicates_beta1)}")
print(f"Number of duplicate rsID in beta2_df: {len(duplicates_beta2)}")

In [17]:
beta1_df = beta1_df[beta1_df['rsID'].notna()]
beta1_df = beta1_df.drop_duplicates(subset=["rsID"])

In [18]:
beta2_df = beta2_df[beta2_df['rsID'].notna()]
beta2_df = beta2_df.drop_duplicates(subset=["rsID"])

In [ ]:
print(f'lenght df1: {len(df1)}')
print(f'lenght df2: {len(df2)}')

In [ ]:
print(f'lenght df1: {len(beta1_df)}')
print(f'lenght df2: {len(beta2_df)}')

In [21]:
# Merge the beta values on SNP ID to ensure we're correlating the same variants
# merged_betas = pd.merge(beta1_df, beta2_df, on="rsID")
merged_betas = beta1_df.merge(beta2_df, on="rsID")

In [22]:
merged_betas = merged_betas[(merged_betas["P1"] > 0.0) & (merged_betas["P2"] > 0.0)]

In [ ]:
merged_betas.head()

In [ ]:
print(f'lenght merged: {len(merged_betas)}')

In [ ]:
suggestive_merged_betas = suggestive_snps.merge(merged_betas, on="SNPID", how="left")
# suggestive_merged_betas = merged_betas[(merged_betas["P1"] < 0.001) & (merged_betas["P2"] < 0.001)]
suggestive_merged_betas

In [ ]:
print(f'length merged: {len(suggestive_merged_betas)}')

In [ ]:
from scipy.stats import pearsonr

# Calculate the Pearson correlation between the beta values
correlation, p_value = pearsonr(merged_betas["BETA1"], merged_betas["BETA2"])

print(f"Pearson correlation: {correlation}")
print(f"P-value: {p_value}")


In [ ]:
from scipy.stats import pearsonr

# Calculate the Pearson correlation between the beta values
correlation, p_value = pearsonr(suggestive_merged_betas["BETA1"], suggestive_merged_betas["BETA2"])

print(f"Pearson correlation: {correlation}")
print(f"P-value: {p_value}")


In [29]:
merged_betas.loc[merged_betas['BETA1'] > 0, 'BETA1_bin'] = 1
merged_betas.loc[merged_betas['BETA1'] <= 0, 'BETA1_bin'] = -1

merged_betas.loc[merged_betas['BETA2'] > 0, 'BETA2_bin'] = 1
merged_betas.loc[merged_betas['BETA2'] <= 0, 'BETA2_bin'] = -1

In [ ]:
jaccard_score(merged_betas["BETA1_bin"], merged_betas["BETA2_bin"])

In [ ]:
from scipy.stats import pearsonr

# Calculate the Pearson correlation between the beta values
correlation, p_value = pearsonr(merged_betas["BETA1_bin"], merged_betas["BETA2_bin"])

print(f"Pearson correlation: {correlation}")
print(f"P-value: {p_value}")


In [ ]:
merged_betas

In [ ]:
def calculate_grouped_correlation(df, bin_column, bins=5):
    # Create the bins dynamically
    df[f'{bin_column}_bins'] = pd.cut(df[bin_column], bins=bins)
    
    # Apply groupby and calculate both the correlation and the count of values in each bin
    results = df.groupby(f'{bin_column}_bins').apply(
        lambda x: pd.Series({
            'correlation': x[['BETA1', 'BETA2']].corr(method='pearson').iloc[0, 1],
            'binned_correlation' : x[['BETA1_bin', 'BETA2_bin']].corr(method='pearson').iloc[0, 1],
            'binned_jaccard' : jaccard_score(x["BETA1_bin"], x["BETA2_bin"]),
            'count': x.shape[0]
        })
    )
    
    return results

# Example: Group by P1 with 10 bins and show correlation and count
# grouped_corr = calculate_grouped_correlation(merged_betas, 'P1', bins=10)
grouped_corr = calculate_grouped_correlation(merged_betas, 'P2', bins=50)
print(grouped_corr)

In [34]:
keep_merged_beta = merged_betas[(merged_betas["P1"] < 0.02) & (merged_betas["P2"] < 0.02)]

In [ ]:
# Create a sign matrix where each value is 1, -1, or 0
sign_matrix = np.sign(keep_merged_beta[["BETA1", "BETA2"]])

# Check if all beta values in each row are pointing in the same direction
same_direction = (sign_matrix.nunique(axis=1) == 1)

# Add the result as a new column to the DataFrame
keep_merged_beta['same_direction'] = same_direction

print(keep_merged_beta)

In [ ]:
counted = keep_merged_beta["same_direction"].value_counts()
counted_true = keep_merged_beta["same_direction"].value_counts().tolist()[0]
counted_false = keep_merged_beta["same_direction"].value_counts().tolist()[1]
counted

In [ ]:
print(f"Beta values in same direction: {(counted_true/len(keep_merged_beta)):.2f}%")
print(f"Beta values in same direction: {(counted_false/len(keep_merged_beta)):.2f}%")



In [ ]:
keep_merged_beta["same_direction"].value_counts().tolist()[0]


In [ ]:
# Calculate the correlation matrix of the beta columns
correlation_matrix = keep_merged_beta[["BETA1", "BETA2"]].corr()

# Extract the upper triangle of the correlation matrix, excluding the diagonal
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Calculate the mean of the upper triangle correlations
mean_correlation = upper_triangle.stack().mean()

print("Average pairwise correlation:", mean_correlation)

In [ ]:
keep_merged_beta[["BETA1", "BETA2"]].corr()

In [81]:
suggestive_merged_betas = suggestive_merged_betas.reset_index()

In [82]:
from Bio import Entrez
import pandas as pd
from easy_entrez import EntrezAPI
import myvariant
import json
from types import SimpleNamespace

data = '{"name": "John Smith", "hometown": {"name": "New York", "id": 123}}'

# Your list of RSIDs
rsids = suggestive_merged_betas['rsID']  # Example RSIDs

# Function to fetch gene symbols from NCBI
def fetch_gene_symbol(rsid):

    mv = myvariant.MyVariantInfo()
    query_out = mv.query(rsid, scopes='dbsnp.rsid', fields='dbsnp')
    # print(query_out)
    if 'gene' in query_out["hits"][0]["dbsnp"]:
        if "symbol" in query_out["hits"][0]["dbsnp"]["gene"]:
            symbol = query_out["hits"][0]["dbsnp"]["gene"]["symbol"]
            # print(symbol)
            return symbol
        else:
            return rsid
    else:
        return rsid

    # entrez_api = EntrezAPI('Project name', 'your@mail.com')
    # rs6311 = entrez_api.fetch([rsid], max_results=1, database='snp').data
    # namespaces = {'ns0': 'https://www.ncbi.nlm.nih.gov/SNP/docsum'}
    # genes = [
    #     name.text
    #     for name in rs6311.findall('.//ns0:GENE_E/ns0:NAME', namespaces)
    # ]
    # print(genes)
    # return genes

# Create a DataFrame to store results
# results = pd.DataFrame({'RSID': rsids, 'GeneSymbol': [fetch_gene_symbol(rsid) for rsid in rsids]})

# print(results)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from adjustText import adjust_text  # Import for handling text overlaps

# Example dataframe columns
BETA1 = suggestive_merged_betas['BETA1']
BETA2 = suggestive_merged_betas['BETA2']
P1 = suggestive_merged_betas['P1']  # P-values for the first set
P2 = suggestive_merged_betas['P2']  # P-values for the second set
AnnotationColumn = suggestive_merged_betas['rsID']  # The column used for annotations

# For the size of the bubbles, we use the -log10(p-values) to emphasize smaller p-values
p_together = -np.log10((P1 + P2) / 2)
bubble_size = p_together
# Small value to avoid division by zero
epsilon = 1e-10
# Calculate the closeness
p_closeness = 1 - (np.abs(P1 - P2) / (np.maximum(P1, P2) + epsilon))
# bubble_size = p_closeness

# Normalize bubble size to a reasonable range for plotting
bubble_size = 200 * (bubble_size - bubble_size.min()) / (bubble_size.max() - bubble_size.min()) + 20

# Define a threshold for high values (adjust as needed)
threshold = np.mean(p_together) + 2 * np.std(p_together)
# threshold = 0

# Plotting
plt.figure(figsize=(10, 8))
scatter = plt.scatter(BETA1, BETA2, s=bubble_size, c=p_together, cmap='coolwarm', alpha=0.7, edgecolors="w", linewidth=1)

# Collect annotations in a list for adjustText
annotations = []

# Annotate high values with arrows pointing to the points beneath the text


genes= []
for i in range(len(p_together)):
    if p_together[i] > threshold:
        # gene = fetch_gene_symbol(AnnotationColumn[i])
        gene = suggestive_merged_betas[suggestive_merged_betas["rsID"] == AnnotationColumn[i]]["GENE"].values[0]
        print(f"-log10(p-values): {p_together[i]} -- gene: {gene} -- rsId: {AnnotationColumn[i]}")
        genes.append(gene)
        
        # Set the annotation position slightly above the point
        annotation = plt.annotate(gene, (BETA1[i], BETA2[i]), textcoords="offset points", 
                                  xytext=(1, 1), ha='center', fontsize=9, 
                                  arrowprops=dict(arrowstyle="->", lw=0.5, color='black'))  # Arrow points from text to the point
        annotations.append(annotation)

for i in genes: print(i)


# Add labels and title
plt.xlabel('BETA value IPH Manual')
plt.ylabel('BETA value IPH Model')
plt.title("BETA's of IPH Manual vs IPH Model with Color by -log10(P-values)")

# Add a color bar to indicate the difference in p-values
colorbar = plt.colorbar(scatter, label='-log10((P-value Manual + P-value Model) / 2)')

# Adjust the layout to account for the color bar
plt.tight_layout()

# Automatically adjust the annotations to avoid overlap and align arrows
adjust_text(annotations)

# Calculate correlation coefficient and p-value
correlation_coefficient, p_value = stats.pearsonr(BETA1, BETA2)

# Fit a linear regression line to plot
slope, intercept = np.polyfit(BETA1, BETA2, 1)
regression_line = slope * np.array(BETA1) + intercept

# Plot the regression line
plt.plot(BETA1, regression_line, color='black', linestyle='--', linewidth=1, 
         label=f'y = {slope:.2f}x + {intercept:.2f}')

# Annotate the plot with the correlation coefficient and p-value
plt.text(0.05, 0.95, f'Correlation (r) = {correlation_coefficient:.2f}\nP-value = {p_value:.2e}', 
         transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')

# Show the plot
plt.legend()
plt.grid()
plt.savefig(os.path.join(PLOTS_loc, "BETA_corr_300dpi." + PHENOTYPE + "_vs_IPH_Model.no_genes.png"), format='png', dpi=300)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def calculate_binned_correlation(df, p1_col, p2_col, beta1_col, beta2_col, bins_p1=10, bins_p2=10):
    # Bin the P1 and P2 values using the specified number of bins
    df['COL1_binned'] = pd.cut(df[p1_col], bins=bins_p1)
    df['COL2_binned'] = pd.cut(df[p2_col], bins=bins_p2)
    
    # Get unique bin labels for sorting later
    sorted_p1_bins = df['COL1_binned'].cat.categories
    sorted_p2_bins = df['COL2_binned'].cat.categories[::-1]

    # Create an empty matrix to hold correlations
    correlation_matrix = np.full((len(sorted_p2_bins), len(sorted_p1_bins)), np.nan)

    # Fill the matrix with correlations for each pair of binned P1 and P2
    for i, p2_bin in enumerate(sorted_p2_bins):
        for j, p1_bin in enumerate(sorted_p1_bins):
            # Filter data for specific P1 and P2 bins
            sub_df = df[(df['COL1_binned'] == p1_bin) & (df['COL2_binned'] == p2_bin)]
            
            if i ==0 and j ==0:
                print(f"p1 bin: {p1_bin} -- p2 bin: {p2_bin}")
                print(sub_df)
            # Check if we have enough values to calculate correlation (minimum 3)
            if len(sub_df) >= 3:
                correlation = sub_df[[beta1_col, beta2_col]].corr().iloc[0, 1]  # Get correlation
                correlation_matrix[i, j] = correlation  # Store the correlation
            else:
                # Keep it as NaN if there are fewer than 3 values
                pass
    
    # correlation_matrix = np.flip(correlation_matrix,1) # horizontal flip
    # sorted_p2_bins = sorted_p2_bins.reverse() # reverse the order of list elementsac

    # Plot the heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(correlation_matrix, xticklabels=sorted_p1_bins, yticklabels=sorted_p2_bins, cmap='coolwarm', annot=False, fmt='.1f', linewidth=.5, vmin=-1, vmax=1, cbar_kws={'ticks': [-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1]})
    
    # Get the current tick positions and labels for the x and y axes
    xticks_positions = plt.gca().get_xticks()  # Get current tick positions for the x-axis
    yticks_positions = plt.gca().get_yticks()  # Get current tick positions for the y-axis

    # Get the current tick labels for the x and y axes
    xticks_labels = [item.get_text() for item in plt.gca().get_xticklabels()]
    yticks_labels = [item.get_text() for item in plt.gca().get_yticklabels()]

    # Replace the first label on the x-axis and y-axis
    xticks_labels[0] = '(0.0, 0.02]'
    yticks_labels[-1] = '(0.0, 0.02]'

    # Set the new tick labels back to the plot
    plt.xticks(xticks_positions, xticks_labels)
    plt.yticks(yticks_positions, yticks_labels)

    plt.xlabel('P values IPH Manual')
    plt.ylabel('P values IPH Model')
    plt.title(f'Correlation of Beta\'s between IPH Manual scoring vs IPH Model scoring (Binned by P values)')
    plt.savefig(os.path.join(PLOTS_loc,"BETA_corr_heat_300dpi." + PHENOTYPE + "_vs_IPH_Model.png"), format='png', dpi=300)
    plt.show()

    return correlation_matrix

# Example usage:
# Assume your dataframe is 'merged_betas'
correlation_matrix = calculate_binned_correlation(
    df=merged_betas, 
    p1_col='P1', 
    p2_col='P2', 
    beta1_col='BETA1', 
    beta2_col='BETA2', 
    bins_p1=50,  # Number of bins for P1
    bins_p2=50   # Number of bins for P2
)
